In [10]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix

In [11]:
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
           'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

df = pd.read_csv('pima-indians-diabetes.csv', names = columns)
df.shape


(768, 9)

In [12]:
#let's check if there are any na values in any column

df.isna().any() 

Pregnancies                 False
Glucose                     False
BloodPressure               False
SkinThickness               False
Insulin                     False
BMI                         False
DiabetesPedigreeFunction    False
Age                         False
Outcome                     False
dtype: bool

In [13]:
df[df == 0].head() # there are zeros in data let's replace them by the column mean

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
for col in df.columns[:-1]:
    df[col].replace(to_replace=0, value=np.mean(df[col]), inplace=True)
    
df.head(10) # replaces zero values in features by their column means

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.000000,148.0,72.000000,35.000000,79.799479,33.600000,0.627,50,1
1,1.000000,85.0,66.000000,29.000000,79.799479,26.600000,0.351,31,0
2,8.000000,183.0,64.000000,20.536458,79.799479,23.300000,0.672,32,1
3,1.000000,89.0,66.000000,23.000000,94.000000,28.100000,0.167,21,0
4,3.845052,137.0,40.000000,35.000000,168.000000,43.100000,2.288,33,1
5,5.000000,116.0,74.000000,20.536458,79.799479,25.600000,0.201,30,0
6,3.000000,78.0,50.000000,32.000000,88.000000,31.000000,0.248,26,1
7,10.000000,115.0,69.105469,20.536458,79.799479,35.300000,0.134,29,0
8,2.000000,197.0,70.000000,45.000000,543.000000,30.500000,0.158,53,1
9,8.000000,125.0,96.000000,20.536458,79.799479,31.992578,0.232,54,1


In [15]:
# let's separate the features from labels

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X.shape, y.shape

((768, 8), (768,))

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 0)

X_train.shape

(537, 8)

In [17]:
# no need for label encoding for features and labels
# feature scaling

sc_X = StandardScaler()
sc_X.fit_transform(X_train)
sc_X.transform(X_test)

array([[-1.1461866 ,  2.55688505,  0.29184926, ...,  1.47038609,
         2.78594417, -0.96569189],
       [-0.81582693, -0.48558291,  0.12553806, ...,  0.13971749,
        -0.1876381 , -0.88240283],
       [-0.1551076 , -1.51076233, -0.87232915, ...,  0.19695055,
        -0.22668514, -0.71582471],
       ...,
       [-0.1551076 ,  0.67187772,  1.12340527, ...,  1.627777  ,
         0.53623395, -0.96569189],
       [-0.48546727, -0.18795018,  0.12553806, ..., -0.90478581,
        -1.07971278, -0.79911377],
       [-1.1461866 , -0.48558291, -0.04077314, ..., -0.26091391,
         1.06487079, -0.79911377]])

Use 1 output node. Output 0 (<0.5) is considered class A and 1 (>=0.5) is considered class B (in case of sigmoid)

Use 2 output nodes. The input belongs to the class of the node with the highest value/probability (argmax).

In the second case you are probably writing about softmax activation function. If that's true, than the sigmoid is just a special case of softmax function. That's easy to show.

$$y=1/(1+e^x)=1/(1+1/e^x)=1/((e^x+1)/e^x)=e^x/(1+e^x)=e^x/(e^0+e^x)$$

As you can see sigmoid is the same as softmax. You can think that you have two outputs, but one of them has all weights equal to zero and therefore its output will be always equal to zero.

So the better choice for the binary classification is to use one output unit with sigmoid instead of softmax with two output units, because it will update faster.

In [27]:
EPOCHS = 300
batch_size = 20

def model_baseline():
    model = Sequential()
    model.add(Dense(input_dim = 8, units = 10, activation = 'relu',
                    kernel_initializer = 'uniform'))
    model.add(Dropout(0.1))
    model.add(Dense(units = 10, activation = 'relu',
                    kernel_initializer = 'uniform'))
    model.add(Dropout(0.1))
    model.add(Dense(units = 1, activation = 'sigmoid',
                    kernel_initializer = 'uniform'))
    model.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model    
            
stratified_kfold = StratifiedKFold(n_splits=10)
classifier = KerasClassifier(build_fn = model_baseline, epochs = EPOCHS, batch_size = batch_size)
classifier.fit(X_train, y_train)
              
results = cross_val_score(classifier, X_train, y_train, cv = stratified_kfold, n_jobs = -1, scoring = 'accuracy')
print(results)
print()
print('The mean accuracy is: {}%'.format(np.mean(results) * 100))
print('The variance is: {}'.format(np.var(results)))
              

Epoch 1/300
537/537 [==============================] - 0s 358us/step - loss: 0.6866 - accuracy: 0.6387
Epoch 2/300
537/537 [==============================] - 0s 82us/step - loss: 0.6804 - accuracy: 0.6387
Epoch 3/300
537/537 [==============================] - 0s 82us/step - loss: 0.6736 - accuracy: 0.6387
Epoch 4/300
537/537 [==============================] - 0s 67us/step - loss: 0.6720 - accuracy: 0.6313
Epoch 5/300
537/537 [==============================] - 0s 89us/step - loss: 0.6635 - accuracy: 0.6480
Epoch 6/300
537/537 [==============================] - 0s 89us/step - loss: 0.6565 - accuracy: 0.6536
Epoch 7/300
537/537 [==============================] - 0s 82us/step - loss: 0.6397 - accuracy: 0.6778
Epoch 8/300
537/537 [==============================] - 0s 82us/step - loss: 0.6401 - accuracy: 0.6574
Epoch 9/300
537/537 [==============================] - 0s 67us/step - loss: 0.6315 - accuracy: 0.6629
Epoch 10/300
537/537 [==============================] - 0s 82us/step - loss: 0.62

537/537 [==============================] - 0s 67us/step - loss: 0.5252 - accuracy: 0.7412
Epoch 81/300
537/537 [==============================] - 0s 74us/step - loss: 0.5482 - accuracy: 0.7263
Epoch 82/300
537/537 [==============================] - 0s 67us/step - loss: 0.5434 - accuracy: 0.7356
Epoch 83/300
537/537 [==============================] - 0s 60us/step - loss: 0.5509 - accuracy: 0.7169
Epoch 84/300
537/537 [==============================] - 0s 67us/step - loss: 0.5390 - accuracy: 0.7412
Epoch 85/300
537/537 [==============================] - 0s 67us/step - loss: 0.5334 - accuracy: 0.7356
Epoch 86/300
537/537 [==============================] - 0s 60us/step - loss: 0.5398 - accuracy: 0.7412
Epoch 87/300
537/537 [==============================] - 0s 73us/step - loss: 0.5418 - accuracy: 0.7244
Epoch 88/300
537/537 [==============================] - 0s 69us/step - loss: 0.5461 - accuracy: 0.7169
Epoch 89/300
537/537 [==============================] - 0s 67us/step - loss: 0.5364 - 

537/537 [==============================] - 0s 74us/step - loss: 0.5114 - accuracy: 0.7449
Epoch 160/300
537/537 [==============================] - 0s 74us/step - loss: 0.5158 - accuracy: 0.7430
Epoch 161/300
537/537 [==============================] - 0s 67us/step - loss: 0.5259 - accuracy: 0.7356
Epoch 162/300
537/537 [==============================] - 0s 76us/step - loss: 0.5236 - accuracy: 0.7449
Epoch 163/300
537/537 [==============================] - 0s 74us/step - loss: 0.5172 - accuracy: 0.7337
Epoch 164/300
537/537 [==============================] - 0s 74us/step - loss: 0.5232 - accuracy: 0.7430
Epoch 165/300
537/537 [==============================] - 0s 67us/step - loss: 0.5163 - accuracy: 0.7393
Epoch 166/300
537/537 [==============================] - 0s 74us/step - loss: 0.5231 - accuracy: 0.7393
Epoch 167/300
537/537 [==============================] - 0s 67us/step - loss: 0.5254 - accuracy: 0.7318
Epoch 168/300
537/537 [==============================] - 0s 74us/step - loss: 

537/537 [==============================] - 0s 104us/step - loss: 0.5092 - accuracy: 0.7467
Epoch 238/300
537/537 [==============================] - 0s 74us/step - loss: 0.5013 - accuracy: 0.7523
Epoch 239/300
537/537 [==============================] - 0s 67us/step - loss: 0.5180 - accuracy: 0.7579
Epoch 240/300
537/537 [==============================] - 0s 67us/step - loss: 0.5041 - accuracy: 0.7412
Epoch 241/300
537/537 [==============================] - 0s 71us/step - loss: 0.5186 - accuracy: 0.7579
Epoch 242/300
537/537 [==============================] - 0s 66us/step - loss: 0.5060 - accuracy: 0.7430
Epoch 243/300
537/537 [==============================] - 0s 67us/step - loss: 0.5118 - accuracy: 0.7542
Epoch 244/300
537/537 [==============================] - 0s 75us/step - loss: 0.4995 - accuracy: 0.7449
Epoch 245/300
537/537 [==============================] - 0s 67us/step - loss: 0.5138 - accuracy: 0.7505
Epoch 246/300
537/537 [==============================] - 0s 74us/step - loss:

Tried my best to brute force every hyperparameter manually each time instead running a grid search because I don't think my laptop will get that done in minutes :D

P.S: this is not the best accuracy I got but I just stuck to this one.

In [29]:
predictions = classifier.predict(X_test)
df_pred = pd.Series(predictions.flatten())
df_actual = pd.Series(y_test)

print(df_pred.value_counts())
df_actual.value_counts()

0    184
1     47
dtype: int64


0    157
1     74
dtype: int64

In [30]:
cm = confusion_matrix(y_test, predictions)

cm_df = pd.DataFrame(data=cm, columns=[['predicted','predicted'], ['Not Diabetic','Diabetic']],
                    index=[['actual','actual'], ['Not Diabetic','Diabetic']])

cm_df

predicted         
                    Not Diabetic Diabetic
actual Not Diabetic          138       19
       Diabetic               46       28

In [32]:
test_accuracy = ((139+46)/y_test.shape[0]) * 100
print('The accuracy of predicting the test set is {}%'.format(test_accuracy))

# the accuracy difference between train/test set is high, but why?

The accuracy of predicting the test set is 80.08658008658008%
